# Load the necessary libraries

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
from sklearn.impute import SimpleImputer

ModuleNotFoundError: No module named 'sklearn'

# Load the dataset

In [2]:
file_path = 'D:\FastAPI_ML_Task\data\global-data-on-sustainable-energy.csv'
df = pd.read_csv(file_path)

#### Clean column names

In [3]:
df.columns = df.columns.str.replace('\n', ' ').str.strip()

### Remove rows with missing values in the target variable

In [4]:
target_col = 'Renewable energy share in the total final energy consumption (%)'
df = df.dropna(subset=[target_col])

### Fill missing values in numerical columns with the median

In [5]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='median')
df[num_cols] = imputer.fit_transform(df[num_cols])

### Forward fill for any categorical data (in case needed)

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
print("Remaining missing values:\n", df.isnull().sum())

In [ ]:
df.to_csv('../data/cleaned_energy_data.csv', index=False)

print("Data cleaning complete. Cleaned data saved to 'data/cleaned_energy_data.csv'.")

In [ ]:
# Load the dataset again
df = pd.read_csv('../data/cleaned_energy_data.csv')

# Print column names to check for any inconsistencies
print("Column names in the dataset:")
print(df.columns)


In [ ]:
# Clean the column names by explicitly replacing the newline character using raw string
df.columns = df.columns.str.replace(r'\n', ' ').str.strip()

# Converting the columns to numeric
df['Density (P/Km2)'] = pd.to_numeric(df['Density (P/Km2)'], errors='coerce')
df['Land Area(Km2)'] = pd.to_numeric(df['Land Area(Km2)'], errors='coerce')

# Save the dataset after type conversion
df.to_csv('../data/cleaned_energy_data.csv', index=False)

print("Data type conversion complete. Check data types:")
print(df.dtypes)

In [ ]:
print("Column names in the dataset:")
print(df.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Reload the cleaned data
df = pd.read_csv('../data/cleaned_energy_data.csv')

# Calculate the correlation matrix
corr_matrix = df.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

# Print out the correlation with the target variable to help select features
target_col = 'Renewable energy share in the total final energy consumption (%)'
correlations_with_target = corr_matrix[target_col].sort_values(ascending=False)
print("Correlation of features with the target variable:\n", correlations_with_target)
